# Python implementation of L-Galaxies

This is a playground to test out the possibility of using `python` as an interface into L-Galaxies.

In [ ]:
# Imports of generic python routines

%load_ext autoreload
%autoreload 2

import astropy.constants as c
import astropy.units as u
import gc
import h5py
h5py.enable_ipython_completer()
import numpy as np
import sys

In [ ]:
# Parameters relating to the python code development.
# Parameters relating to the SAM will be set in the input yaml file.

# Location of code
C_DIR='code-C'
PYTHON_DIR='code-python'

# Development limiter
#n_GRAPH=np.inf
n_GRAPH=2

# Debug/testing switch
b_DEBUG=True

# Verbosity
VERBOSITY=4 # 0 - Major program steps only; 1/2 - Major/minor Counters; 3/4/5 - Debugging diags.

# Script parameters
FILE_PARAMETERS='input/input.yml'
b_DISPLAY_PARAMETERS=True

In [ ]:
# Imports of py-galaxies python routines
sys.path.insert(1,PYTHON_DIR)

# The parameter class, used to store run-time parameters
from parameters import C_parameters

# The graph class, used to store graphs for processing
from graphs import C_graph

# The halo class, used to store halo properties
from halos import C_halo

# The halo_output class and methods used to output halos
from halos import C_halo_output

## Functions

Most of the work to be done in external routines, probably to be coded in C for efficiency.

Here we just include the high-level driver routines.

In [ ]:
# These routines will eventually be moved to halos.py.
# For now they are here to help with code development.

def F_check_runtime_parameters(parameters):
    """
    Check for existence of runtime parameters and if missing define them as False.
    """
    try:
        if parameters.b_HOD==True:
            pass
    except:
        parameters.b_HOD=False

def F_process_halos(halos_this_snap,graph,parameters):
    """
    This is the controlling routine for halo processing.
    """
    for halo in halos_this_snap:
        if parameters.verbosity>=4: print('Processing halo ',halo.halo_ID)
        if halo.b_done==True:
            raise RuntimeError('halo '+str(halo.halo_ID)+' in graph '+str(halo.graph_ID)+' already processed.')
        F_set_baryon_fraction(halo,parameters)
        if parameters.b_HOD==True:
            F_set_stellar_fraction(halo,graph,parameters)
        halo.b_done=True
    
def F_update_halos(halos_last_snap,halos_this_snap,graph,parameters):
    """
    Propagate properties from progenitor halos to descendants.
    Done as a push rather than a pull because sharing determined by progenitor.
    """
    """
    Note that there are two offsets here:
    * Entries in the graph lookup tables are offset by desc_start
    * Entries in the halos_this_snap list are offset by halo_ID_offset
    """
    halo_ID_offset=halos_this_snap[0].halo_ID
    for halo in halos_last_snap:
        # First determine what fraction to give to each descendant
        desc_start=halo.desc_start
        desc_end=desc_start+halo.ndesc
        fractions=graph.direct_desc_contribution[desc_start:desc_end]/ \
            np.sum(graph.direct_desc_contribution[desc_start:desc_end])
        # Now loop over descendants, giving it to them
        for i_desc in range(desc_start,desc_end):
            desc_halo_ID=graph.direct_desc_ids[i_desc]
            desc_halo=halos_this_snap[desc_halo_ID-halo_ID_offset]
            assert desc_halo_ID == desc_halo.halo_ID
            if parameters.verbosity>=5: print('Processing descendant',desc_halo_ID)
            desc_halo.mass_from_progenitors+=fractions[i_desc-desc_start]*halo.mass
            if parameters.b_HOD==True:
                desc_halo.mass_stars_from_progenitors+=fractions[i_desc-desc_start]*halo.mass_stars
                desc_halo.mass_stars+=fractions[i_desc-desc_start]*halo.mass_stars # Could be set later

def F_update_parameters(graph_file,parameters):
    for key, value in graph_file['Header'].attrs.items():
        if b_DISPLAY_PARAMETERS: print(key,value)
        exec('parameters.'+key+'=value')
    parameters.n_graph=len(graph_file['graph_lengths'])
    # Put code in here to either copy table of snapshot redshifts/times from graph_file,
    # Or calculate them if that does not exist.
    # Dummy code follows:
    n_snap=62
    parameters.n_snap=n_snap
    parameters.snap_table=np.empty(n_snap,dtype=[('snap_ID',np.int32),('redshift',np.float32),\
                                   ('lookbacktime_in_years',np.float32)])
    for i_snap in range(n_snap):
        t=(n_snap-i_snap-1)/n_snap # Complete and utter fudge
        parameters.snap_table[i_snap]=(i_snap,(1.-t)**(-2./3.)-1.,1.3e10*t) # Complete and utter fudge

# These routines will be replaced by interfaces to existing L-Galaxies routines, 
# written in C and located in code-C/

def F_set_baryon_fraction(halo,parameters):
    halo.mass_baryon=parameters.baryon_fraction*max(halo.mass,halo.mass_from_progenitors)
    
# This one is a for development testing
# It's full of magic numbers; but it's only a fudge, so not putting them in parameter file
def F_set_stellar_fraction(halo,graph,parameters):
    if halo.mass>1e12:
        halo.star_formation_rate=0.
    else:
        max_mass_stars=0.1*halo.mass_baryon
        halo.star_formation_rate=max(0.,(max_mass_stars-halo.mass_stars)/3e9)
    dt=parameters.snap_table['lookbacktime_in_years'][halo.snap_ID-1]- \
        parameters.snap_table['lookbacktime_in_years'][halo.snap_ID]
    halo.mass_stars+=halo.star_formation_rate*dt

## Main routine

### Initialisation

In [ ]:
# Read in all the parameters of the run from the yaml and graph input files.

parameters=C_parameters(FILE_PARAMETERS,VERBOSITY,b_DEBUG)
if b_DISPLAY_PARAMETERS: print(parameters)

# Open graph input file
graph_file=h5py.File(parameters.graph_input_file,'r')

# Update parameters with information from graph_file
F_update_parameters(graph_file,parameters)

# To avoid clumsy try/except clauses, define missing runtime parameters
F_check_runtime_parameters(parameters)

# Create halo output buffer
halo_output=C_halo_output(parameters)

### Loop over graphs, snapshots, halos, implementing the SAM

In [ ]:
# Loop over graphs
for i_graph in range(min(parameters.n_graph,n_GRAPH)):
    if VERBOSITY>=2: print('Processing graph',i_graph)
    graph=C_graph(i_graph,graph_file,parameters)
    
    # Loop over snapshots
    halos_last_snap = None
    for i_snap in graph.generation_id:
        if i_snap == parameters.NO_DATA_INT: 
            assert halos_last_snap == None
            continue
        if VERBOSITY>=3: print('Processing snapshot',i_snap)
            
        # Initialise halo properties
        halos_this_snap=[C_halo(i_graph,i_snap,i_halo,graph,parameters) for i_halo in 
                         graph.generation_start_index[i_snap]+range(graph.generation_length[i_snap])]
        
        # Propagate information from progenitors to this generation
        # Done as a push rather than a pull because sharing determined by progenitor
        if halos_last_snap != None: 
            F_update_halos(halos_last_snap,halos_this_snap,graph,parameters)
            del halos_last_snap
            #gc.collect() # garbage collection -- safe but very slow.
        
        # Process the halos
        F_process_halos(halos_this_snap,graph,parameters)
            
        # Once all halos have been done, output results
        # This could instead be done on a halo-by-halo basis in F_process_halos
        halo_output.append(halos_this_snap,parameters)
            
        # Tidy up
        halos_last_snap=halos_this_snap

    # Free up memory
    del halos_last_snap
    del halos_this_snap



###  Tidy up and exit

In [ ]:
# Flush buffers, close files and exit
halo_output.close()
graph_file.close()